In [369]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import os
# import json
# from pandas.io.json import json_normalize

In [370]:
import pandas as pd
import numpy as np
# Preprocessing
import re
import preprocessor as p
import string
import contractions
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.corpus import stopwords
# Model
from sklearn import model_selection, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
# Pipeline
from sklearn.pipeline import make_pipeline
from joblib import dump

In [371]:
# Download requires corpus
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [372]:
dataset = pd.read_json("../res/dataset3.json")
dataset = dataset.transpose()
dataset.drop(["post_id", "rationales"], axis=1, inplace=True)
dataset.head()

,annotators,post_tokens
1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","[i, dont, think, im, getting, my, baby, them, ..."
1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","[we, cannot, continue, calling, ourselves, fem..."
1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...","[nawt, yall, niggers, ignoring, me]"
1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[<user>, i, am, bit, confused, coz, chinese, p..."
1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[this, bitch, in, whataburger, eating, a, burg..."


In [373]:
# Create preprocessing function
def preprocess(text):
    # twitter-presprocessor
    text = p.clean(text)
    # lower case
    text = text.lower()
    # Expand Contractions
    expanded = []
    for word in text.split():
        expanded.append(contractions.fix(word, slang=True))
    text = ' '.join(expanded)
    # Remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    text = word_tokenize(text)
    
    lemmanized = preprocess_part2(text)

    # return str(text)
    return lemmanized

def preprocess_part2(text):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(text):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords_updated and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    return str(Final_words)


<h2>Preprocess post_tokens</h2>

In [374]:
bracketed_words = set()
# Regex pattern to match anything that is eclosed by angled brackets
pattern = re.compile("\<(.*?)\>")
# Get all unique bracketed tokens
for e in dataset["post_tokens"]:
    for t in e:
        if ">" in t and "<" in t:
            bracketed_words.add(t)
# Remove all bracketed tokens in post_tokens
dataset["post_tokens"] = [list(filter(lambda token: token not in bracketed_words, entry)) for entry in dataset["post_tokens"]]

In [375]:
# Expand contractions
dataset["post_tokens"] = [[contractions.fix(token, slang=True) for token in entry] for entry in dataset["post_tokens"]]

In [376]:
# Create new column for concatenated text
dataset["text"] = [' '.join(entry) for entry in dataset["post_tokens"]]
# Convert to lowercase
dataset["text"] = [entry.lower() for entry in dataset["text"]]
# Remove punctuation marks
dataset["text"] = [entry.translate(str.maketrans('', '', string.punctuation)) for entry in dataset["text"]]
# Tokenization
dataset['text']= [word_tokenize(entry) for entry in dataset['text']]
# Turn to String
dataset['text']= [str(entry) for entry in dataset['text']]

In [377]:
dataset.drop(["post_tokens"], axis=1, inplace=True)

In [378]:
dataset.head()

,annotators,text
1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","['i', 'do', 'not', 'think', 'i', 'am', 'gettin..."
1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...","['we', 'can', 'not', 'continue', 'calling', 'o..."
1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...","['nawt', 'you', 'all', 'niggers', 'ignoring', ..."
1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","['i', 'am', 'bit', 'confused', 'coz', 'chinese..."
1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","['this', 'bitch', 'in', 'whataburger', 'eating..."


In [426]:
str(dataset.annotators.iloc[1])

"[{'label': 'normal', 'annotator_id': 1, 'target': ['None']}, {'label': 'normal', 'annotator_id': 2, 'target': ['None']}, {'label': 'normal', 'annotator_id': 3, 'target': ['None']}]"

In [500]:
def det_hate(text):
    return 1 if "hatespeech" in text else 0
def det_normal(text):
    return 1 if "normal" in text and "hatespeech" not in text else 0
def det_offensive(text):
    return 1 if "offensive" in text else 0

dataset["HATE"] = dataset.apply(lambda e: det_hate(str(e[0])), axis = 1)
dataset["NOT"] = dataset.apply(lambda e: det_normal(str(e[0])), axis = 1)
dataset["OFFN"] = dataset.apply(lambda e: det_offensive(str(e[0])), axis = 1)

In [ ]:
# Create new column for concatenated text
dataset["text"] = [' '.join(entry) for entry in dataset["post_tokens"]]
# Convert to lowercase
dataset["text"] = [entry.lower() for entry in dataset["text"]]
# Remove punctuation marks
dataset["text"] = [entry.translate(str.maketrans('', '', string.punctuation)) for entry in dataset["text"]]
# Tokenization
dataset['text']= [word_tokenize(entry) for entry in dataset['text']]
# Turn to String
dataset['text']= [str(entry) for entry in dataset['text']]

In [446]:
# Validate if there are rows with HATE 1 and NOT 1
dataset1 = dataset[dataset["HATE"].ne(0) & dataset["NOT"].ne(0)]
len(dataset1)

0

In [524]:
# Get collection of obscene words
obscene_words = []
# Read file
with open('../res/obscene-words.csv') as file: 
    for line in file:
        obscene_words.append(line.rstrip("\n"))
obscene_words = obscene_words[0].split(", ")

# Remove containing numbers
obscene_words = [x for x in obscene_words if not any(c.isdigit() for c in x)]
# Remove containing special characters
special_characters = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
def has_special(text):
    return True if any(c in special_characters for c in text) else 0
obscene_words = list(filter(lambda x: not has_special(x), obscene_words))
# Remove spaces
obscene_words = list(filter(lambda x: " " not in x, obscene_words))
len(obscene_words)

3582

In [526]:
def det_prof(text):
    return 1 if any(badword in text for badword in obscene_words) else 0

dataset["PRFN"] = dataset.apply(lambda e: det_prof(str(e[1])), axis = 1)

In [527]:
dataset["PRFN"].value_counts()

1    17269
0     2879
Name: PRFN, dtype: int64

In [379]:
get_dict = pd.DataFrame.from_dict(dataset["annotators"])
get_dict

,annotators
1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe..."
1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe..."
1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe..."
1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't..."
1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't..."
...,...
9989999_gab,"[{'label': 'offensive', 'annotator_id': 217, '..."
9990225_gab,"[{'label': 'offensive', 'annotator_id': 220, '..."
9991681_gab,"[{'label': 'offensive', 'annotator_id': 206, '..."
9992513_gab,"[{'label': 'hatespeech', 'annotator_id': 209, ..."


In [401]:
list_ = dataset["annotators"].to_list()
list_[0:5]
len(list_)

20148

In [402]:
flatten = [val for sublist in list_ for val in sublist]
flatten[0:5]
len(flatten)

60444

In [410]:
annotator = pd.DataFrame(flatten)
# annotator.drop(["annotator_id"], axis=1, inplace=True)
annotator

,label,annotator_id,target
0,normal,1,[None]
1,normal,2,[None]
2,normal,3,[None]
3,normal,1,[None]
4,normal,2,[None]
...,...,...,...
60439,hatespeech,253,[Asian]
60440,offensive,222,[Asian]
60441,hatespeech,200,"[African, Islam]"
60442,offensive,202,"[Islam, Jewish]"


In [412]:
len(annotator.annotator_id.value_counts())

253

In [396]:
annotator.label.value_counts()

normal        24449
hatespeech    18070
offensive     17925
Name: label, dtype: int64

In [399]:
len(dataset)

20148

In [383]:
#annotator = dataset.drop(dataset.columns[[0,2]], axis = 1 ).join(annotator)

In [384]:
annotator['target'].explode().value_counts()

None            21314
African         10015
Islam            6686
Women            6126
Jewish           5961
Homosexual       5617
Other            4708
Refugee          3100
Arab             2907
Caucasian        2359
Men              1758
Asian            1350
Hispanic         1299
Christian         270
Disability        221
Minority          188
Heterosexual      141
Economic           90
Hindu              89
Nonreligious       87
Indigenous         83
Indian             81
Buddhism           12
Bisexual            8
Asexual             5
Name: target, dtype: int64

In [385]:
annotator['label'].explode().value_counts()

normal        24449
hatespeech    18070
offensive     17925
Name: label, dtype: int64

In [386]:
label = annotator['label']
annotator = annotator.join(label.str.get_dummies().astype('int32'))
annotator

,label,annotator_id,target,hatespeech,normal,offensive
0,normal,1,[None],0,1,0
1,normal,2,[None],0,1,0
2,normal,3,[None],0,1,0
3,normal,1,[None],0,1,0
4,normal,2,[None],0,1,0
...,...,...,...,...,...,...
60439,hatespeech,253,[Asian],1,0,0
60440,offensive,222,[Asian],0,0,1
60441,hatespeech,200,"[African, Islam]",1,0,0
60442,offensive,202,"[Islam, Jewish]",0,0,1


In [387]:
#Race = African, Arab, Caucasian, Asian, Hispanic, Indian
#Religion = Islam, Jewish, Christian, Hindu, Nonreligious, Buddhism
#Gender = Women, Homosexual, Men, Heterosexual, Bisexual, Asexual
#Other = Other, Refugee, Disability, Minority, Economic, Indigenous
#None = None

#Race = annotator.loc[annotator['target'].isin(['African', 'Arab', 'Caucasian', 'Asian', 'Hispanic', 'Indian'])]
#Race.head()